Instalación de la libreia deap, si requiere mayor información consultar:

https://deap.readthedocs.io/en/master/

In [1]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.6 MB/s eta 0:00:00


In [2]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import numpy as np


In [114]:

# Matriz sep
sep = np.array([[1, 1.5, 2],
                [1, 1.5, 1.5],
                [1, 1, 1]])

# Matriz tels
tels = np.array([[1, 2, 3, 4, 5],
                 [2, 3, 4, 5, 6],
                 [3, 4, 5, 6, 7]])

Función para crear el individuo, el argumento size se utiliza para establecer el tamaño de los dos arreglos que componene el individuo, puede agregar mas argumentos,


tecuerde que necesitamos genera individuos con la siguiente estructura:

[[v5,v6,v3,v1,v4,v2],[p1,p2,p1,p2,p3,p2]]

donde Vi es el indice del codigo del vuelo y
p1 es el indice o número de pista donde aterriza el avion.

un individuo valido seria 

[[1,2,3,5,4,6],[1,2,1,2,3,2]]

Del individuo determino que el vuelo 6 aterriza en la pista 2


In [3]:
def initIndividual(icls, size):
    a=np.random.randint(5,size=size)
    b=np.random.randint(10,size=size)
    indv=np.stack((a, b))
    return icls(indv)

In [4]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

In [5]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Mucha atención en la creación del individuo se define que es numpy ndarray

creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

toolbox = base.Toolbox()


#Se define que la creación del individuo de realizará con la función initIndividual
#colocar especial atención al número 5 al final de la función 
#este es el valor del parámetro size en la función de creación del individuo initIndividual
#Si definió más parámetros debe pasarlos

toolbox.register("individual_guess", initIndividual, creator.Individual,5)

#Crea la población utilizando el creador de individuos individual_guess anteriormente registrado
toolbox.register("population", tools.initRepeat, list, toolbox.individual_guess)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Plantilla para programar la función de evaluación,** no eliminar la coma al final de la función es necesaria para el parse por parte del algoritmo** por el momento esta funcion solo devuelve un valor aleatorio adecuar al problema

Habilitar las impresiones para visualizar la estructura del individuo

In [143]:
def evalOneMax(individual):
    flight_order = []
    fitness = 0

    for i, flight_type in enumerate(individual[:, 0]):
        flight_id = int(flight_type)

        if i >= len(tels) or flight_id >= len(tels[0]):
            break  # Salir del ciclo si i o flight_id están fuera de los límites de tels

        if i > 0:
            prev_tels = [tels[r][flight_id] + sep_time for r in range(i-1)]
            tla = max(tels[i][flight_id], max(prev_tels, default=0))
        else:
            tla = tels[i][flight_id]

        delay = tla - tels[i][flight_id]
        fitness += delay ** 2

        flight_order.append(flight_id)

    return fitness,


Plantilla de la función de cruce en esta pagina puede ver ejemplos de implementación de operadores de cruce, tenga en cuenta que para este problema el individuo es np.ndarray

https://github.com/DEAP/deap/blob/master/deap/tools/crossover.py

In [144]:
def cxTwoPointCopy(ind1, ind2):
    size = len(ind1)
    cxpoint1 = np.random.randint(size)
    cxpoint2 = np.random.randint(size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]
    return ind1, ind2

Plantilla de la función de mutación en esta pagina puede ver ejemplos de implementación de operadores de mutacion, tenga en cuenta que para este problema el individuo es np.ndarray

https://github.com/DEAP/deap/blob/master/deap/tools/mutation.py

**La , al final de la función es necesaria para el parse del algoritmo**



In [179]:
def mutFlipBit(ind1, mutpb):
    # Mutación por flip de bits
    for i in range(len(ind1)):
        if random.random() < mutpb:
            ind1[i] = np.logical_not(ind1[i])
    return ind1,

Se establecen los operadores del algorimo evolutivo utilizando las funciones correspondientes anteriormente establecidas

In [180]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy)
toolbox.register("mutate", mutFlipBit,mutpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

Se crea y ejecuta el algoritmo evolutivo

In [186]:
def main():
    random.seed(64)

    pop = toolbox.population(n=300)

    # Numpy equality function (operators.eq) between two arrays returns the
    # equality element wise, which raises an exception in the if similar()
    # check of the hall of fame. Using a different equality function like
    # numpy.array_equal or numpy.allclose solve this issue.
    hof = tools.HallOfFame(1, similar=np.array_equal)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats,
                        halloffame=hof)

    return pop, stats, hof

if __name__ == "__main__":
   p,l,h=main()
   print(h)

gen	nevals	avg	std	min	max
0  	300   	0  	0  	0  	0  
1  	176   	0  	0  	0  	0  
2  	160   	0  	0  	0  	0  
3  	190   	0  	0  	0  	0  
4  	171   	0  	0  	0  	0  
5  	145   	0  	0  	0  	0  
6  	191   	0  	0  	0  	0  
7  	171   	0  	0  	0  	0  
8  	176   	0  	0  	0  	0  
9  	179   	0  	0  	0  	0  
10 	186   	0  	0  	0  	0  
11 	199   	0  	0  	0  	0  
12 	180   	0  	0  	0  	0  
13 	184   	0  	0  	0  	0  
14 	181   	0  	0  	0  	0  
15 	175   	0  	0  	0  	0  
16 	172   	0  	0  	0  	0  
17 	174   	0  	0  	0  	0  
18 	186   	0  	0  	0  	0  
19 	182   	0  	0  	0  	0  
20 	166   	0  	0  	0  	0  
21 	183   	0  	0  	0  	0  
22 	178   	0  	0  	0  	0  
23 	189   	0  	0  	0  	0  
24 	191   	0  	0  	0  	0  
25 	164   	0  	0  	0  	0  
26 	189   	0  	0  	0  	0  
27 	192   	0  	0  	0  	0  
28 	172   	0  	0  	0  	0  
29 	196   	0  	0  	0  	0  
30 	170   	0  	0  	0  	0  
31 	158   	0  	0  	0  	0  
32 	182   	0  	0  	0  	0  
33 	184   	0  	0  	0  	0  
34 	182   	0  	0  	0  	0  
35 	167   	0  	0  	0  	0  
3